**Import libraries**

In [1]:
import ast
import pandas as pd

**Read data**

In [2]:
data = pd.read_csv("./data/delivery_info.csv")
data.head()

,id,total_review,avg,display_total_review,city_id,restaurant_id,name,categories,cuisines,address,min_order_value,price_range,is_quality_merchant,latitude,longitude,brand_id,brand_name,open_time,close_time,url
0,2006,500,4.6,500+,218,67319,Luna Steak & Pasta,Quán ăn,Món Âu,"236 Đội Cấn, Ba Đình, Hà Nội","{'resource_name': 'delivery_order_minimum', 'r...","{'resource_name': 'avg_price_format', 'resourc...",False,21.035283,105.819824,0,NaN,09:30:00,21:45:59,https://shopeefood.vn/ha-noi/luna-steak-pasta
1,2027,100,4.8,100+,218,5411,Mochi Sweets - Bùi Thị Xuân,Tiệm bánh,"Món Nhật, Món Á","148 Bùi Thị Xuân, P. Bùi Thị Xuân, Hai Bà Trưn...","{'resource_name': 'delivery_order_minimum', 'r...","{'resource_name': 'avg_price_format', 'resourc...",False,21.012153,105.849789,2927,Bánh Mochi Sweets - Hà Nội,09:00:00,22:00:59,https://shopeefood.vn/ha-noi/mochi-sweets-bui-...
2,2052,1000,4.6,999+,218,88765,Cơm Rang Ngõ Tràng Tiền,Quán ăn,Món Việt,"8 Ngõ Tràng Tiền, P. Tràng Tiền, Hoàn Kiếm, Hà...","{'resource_name': 'delivery_order_minimum', 'r...","{'resource_name': 'avg_price_format', 'resourc...",True,21.024030,105.856481,0,NaN,07:00:00,14:00:59,https://shopeefood.vn/ha-noi/com-rang-ngo-tran...
3,2067,1000,4.9,999+,218,134242,Bé Bự Quán - Đặng Văn Ngữ,Quán ăn,Món Việt,"79 - 81 Đặng Văn Ngữ, Đống Đa, Hà Nội","{'resource_name': 'delivery_order_minimum', 'r...","{'resource_name': 'avg_price_format', 'resourc...",False,21.009999,105.831150,0,NaN,07:30:00,20:37:00,https://shopeefood.vn/ha-noi/be-bu-quan-dang-v...
4,2090,4,0.0,4,217,176862,Út Nhỏ - Bông Lan Trứng Muối,Tiệm bánh,Món Việt,"95 Đất Thánh, P. 6, Tân Bình, TP. HCM","{'resource_name': 'delivery_order_minimum', 'r...","{'resource_name': 'avg_price_format', 'resourc...",False,10.783244,106.658182,0,NaN,08:00:00,21:00:00,https://shopeefood.vn/ho-chi-minh/an-vat-ut-nho


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218641 entries, 0 to 218640
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    218641 non-null  int64  
 1   total_review          218641 non-null  int64  
 2   avg                   218641 non-null  float64
 3   display_total_review  218641 non-null  object 
 4   city_id               218641 non-null  int64  
 5   restaurant_id         218641 non-null  int64  
 6   name                  218641 non-null  object 
 7   categories            218640 non-null  object 
 8   cuisines              112358 non-null  object 
 9   address               218641 non-null  object 
 10  min_order_value       218641 non-null  object 
 11  price_range           218641 non-null  object 
 12  is_quality_merchant   218641 non-null  bool   
 13  latitude              218641 non-null  float64
 14  longitude             218641 non-null  float64
 15  

**Filter rows**

In [4]:
data = data[data["city_id"] == 217]

**Drop columns**

In [5]:
data = data.drop(columns=["display_total_review", "city_id", "min_order_value", "open_time", "close_time"])

**Create new columns**

In [6]:
data["district"] = data["address"].map(lambda item: item.split(", ")[-2])
data = data.drop(columns="address")

In [7]:
data["avg_price"] = data["price_range"].map(lambda item: int(ast.literal_eval(item)["resource_args"][0][:2]))
data = data.drop(columns="price_range")

In [8]:
n_categories = data["categories"].value_counts()
mask = n_categories == 1
data = data[(data["categories"] != "Bank - ATM") & (data["categories"] != "Rạp chiếu phim")]
data = data[~data["categories"].isin(n_categories[mask.values].index)]

# data["category_1"] = data["categories"].map(lambda item: item.split(", ")[0])
# data["category_2"] = data["categories"].map(lambda item: item.split(", ")[1] if len(item.split(", ")) == 2 else "Unknown")

# data = data.drop(columns="categories")

**Fill missing values**

In [9]:
data["brand_name"] = data["brand_name"].fillna("Unknown")

In [10]:
data["cuisines"] = data["cuisines"].fillna("Unknown")

**Save to csv file**

In [11]:
data.to_csv("./data/delivery_info_clean.csv", index=False)